# Estimar esforço em desenvolvimento de software com redes neurais profundas

Desenvolvimento do Trabalho de Graduação


## Dependências

- Python 3.6
- numpy >= 1.15.4
- pandas >= 0.23.4
- matplotlib >= 2.2.3

In [246]:
from glob import glob
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras

## Constantes globais de configurações

In [247]:
caminho_dados = 'dados/cocomo/'
dim_inicial = 3
dim_intermediaria = 2

## Carregamento e pre-processamento dos dados

In [270]:
def carregar_tudo(caminho_dados):
    dados = {}
    for arquivo in glob(caminho_dados+'/*.csv'):
        ticker = arquivo.split('/')[-1].split('.')[0]
        tabela = pd.read_csv(arquivo, skipinitialspace=True)
        dados[ticker] = tabela
    return dados

dados = carregar_tudo(caminho_dados)
dados.keys()

dict_keys(['cocomo_nasa2', 'cocomo_sdr', 'cocomo_nasa2000', 'cocomo81', 'cocomo_nasa'])

In [27]:
dados['cocomo_sdr'].keys()

Index(['ProjectID', 'PREC', 'FLEX', 'RESL', 'TEAM', 'PMAT', 'RELY', 'DATA',
       'CPLX', 'RUSE', 'DOCU', 'TIME', 'STOR', 'PVOL', 'ACAP', 'PCAP', 'PCON',
       'AEXP', 'PEXP', 'LTEX', 'TOOL', 'SITE', 'SCED', 'LOC', 'ACTUAL EFFORT'],
      dtype='object')

In [23]:
sorted(['RELY', ' DATA', ' CPLX', ' TIME', ' STOR', ' VIRT', ' TURN', ' ACAP',
       ' AEXP', ' PCAP', ' VEXP', ' LEXP', ' MODP', ' TOOL', ' SCED', ' LOC'])

[' ACAP',
 ' AEXP',
 ' CPLX',
 ' DATA',
 ' LEXP',
 ' LOC',
 ' MODP',
 ' PCAP',
 ' SCED',
 ' STOR',
 ' TIME',
 ' TOOL',
 ' TURN',
 ' VEXP',
 ' VIRT',
 'RELY']

In [28]:
sorted(['PREC', 'FLEX', 'RESL', 'TEAM', 'PMAT', 'RELY', 'DATA',
       'CPLX', 'RUSE', 'DOCU', 'TIME', 'STOR', 'PVOL', 'ACAP', 'PCAP', 'PCON',
       'AEXP', 'PEXP', 'LTEX', 'TOOL', 'SITE', 'SCED', 'LOC', 'ACTUAL EFFORT'])

['ACAP',
 'ACTUAL EFFORT',
 'AEXP',
 'CPLX',
 'DATA',
 'DOCU',
 'FLEX',
 'LOC',
 'LTEX',
 'PCAP',
 'PCON',
 'PEXP',
 'PMAT',
 'PREC',
 'PVOL',
 'RELY',
 'RESL',
 'RUSE',
 'SCED',
 'SITE',
 'STOR',
 'TEAM',
 'TIME',
 'TOOL']

In [274]:
dados['cocomo81']

,ID,PREC,FLEX,RESL,TEAM,PMAT,RELY,DATA,CPLX,RUSE,...,APEX,PLEX,LTEX,TOOL,SITE,SCED,KLOC,Effort,Defects,Months
0,1,h,h,h,vh,vl,l,vh,vl,n,...,l,l,n,vl,h,n,113,2040.0,13027,38.4
1,2,h,h,h,vh,vl,l,vh,l,n,...,h,h,h,vl,h,n,293,1600.0,25229,48.6
2,3,h,h,h,vh,n,n,vh,l,n,...,vh,h,h,l,h,n,132,243.0,3694,28.7
3,4,h,h,h,vh,vl,vl,vh,vl,n,...,h,n,h,vl,h,n,60,240.0,5688,28.0
4,5,h,h,h,vh,vl,l,l,n,n,...,n,h,h,vl,h,n,16,33.0,970,14.3
5,6,h,h,h,vh,vl,vl,n,l,n,...,n,h,h,vl,h,n,4,43.0,553,11.6
6,7,h,h,h,vh,n,vl,n,n,n,...,n,h,h,l,h,n,6.9,8.0,350,10.3
7,8,h,h,h,vh,vl,h,l,vh,n,...,h,vl,vl,vl,h,l,22,1075.0,3511,24.5
8,9,h,h,h,vh,n,h,l,vh,n,...,n,l,l,vl,h,n,30,423.0,1989,24.1
9,10,h,h,h,vh,l,vh,l,vh,n,...,vh,h,n,vl,h,n,29,321.0,1496,23.2


In [271]:
dados['cocomo_nasa']

,RELY,DATA,CPLX,TIME,STOR,VIRT,TURN,ACAP,AEXP,PCAP,VEXP,LEXP,MODP,TOOL,SCED,KLOC,Effort
0,n,h,vh,n,n,l,n,h,n,vh,l,n,h,n,l,70.0,278.0
1,vh,h,h,vh,vh,n,n,vh,vh,vh,n,h,h,h,l,227.0,1181.0
2,n,h,h,vh,h,l,h,h,n,h,l,h,h,n,l,177.9,1248.0
3,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,115.8,480.0
4,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,29.5,120.0
5,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,19.7,60.0
6,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,66.6,300.0
7,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,5.5,18.0
8,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,10.4,50.0
9,h,l,h,n,n,l,l,n,n,n,n,h,h,n,l,14.0,60.0


In [272]:
dados['cocomo_nasa2']

,ID,projectname,cat2,forg,center,year,mode,RELY,DATA,CPLX,...,ACAP,AEXP,PCAP,VEXP,LEXP,MODP,TOOL,SCED,KLOC,Effort
0,1,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,25.90,117.6
1,2,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,24.60,117.6
2,3,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,7.70,31.2
3,4,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,8.20,36.0
4,5,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,9.70,25.2
5,6,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,2.20,8.4
6,7,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,3.50,10.8
7,8,erb,avionicsmonitoring,g,2,1982,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,66.60,352.8
8,9,gal,missionplanning,g,1,1980,semidetached,h,l,h,...,h,h,h,n,h,h,h,n,7.50,72.0
9,10,gal,missionplanning,g,1,1980,semidetached,n,l,h,...,h,vh,vh,n,h,n,n,n,20.00,72.0


In [275]:
dados['cocomo_sdr']

,ID,PREC,FLEX,RESL,TEAM,PMAT,RELY,DATA,CPLX,RUSE,...,PCAP,PCON,AEXP,PEXP,LTEX,TOOL,SITE,SCED,KLOC,Effort
0,1,vh,n,l,h,vl,n,n,l,l,...,vh,vh,vh,h,h,h,l,n,3.000,1.2
1,2,vh,n,l,vh,vl,h,n,l,n,...,h,vh,vh,vh,vh,h,vl,h,2.000,2.0
2,3,xh,n,l,h,vl,h,vh,l,n,...,h,vh,vh,vh,vh,h,vl,h,4.250,4.5
3,4,xh,h,h,h,n,h,h,vh,l,...,h,vl,h,vh,h,n,h,h,10.000,3.0
4,5,n,n,l,h,n,l,n,vh,n,...,h,h,n,n,n,n,h,l,15.000,4.0
5,6,vh,l,xh,xh,h,l,l,n,l,...,vh,h,n,h,h,vl,xh,l,40.530,22.0
6,7,xh,vh,xh,n,h,l,l,l,n,...,vh,vh,n,vh,vh,vl,h,l,4.050,2.0
7,8,n,vl,vh,n,l,l,vh,vl,h,...,h,vh,n,n,n,n,vh,n,31.845,5.0
8,9,vl,vl,n,vl,l,vl,n,l,vh,...,h,vh,h,h,h,n,h,n,114.280,18.0
9,10,n,vl,vh,l,l,vl,h,n,vh,...,vh,vh,h,vh,h,n,n,l,23.106,4.0


## Criação do modelo COCOMO neural

### Tratamento dos parâmetros do COCOMO 2000

In [249]:
params = pd.read_csv('dados/params-cocomo2000.csv', index_col=0)
params

,Symbol,VL,L,N,H,VH,XH
Driver,,,,,,,
PREC,SF1,6.20,4.96,3.72,2.48,1.24,0.00
FLEX,SF2,5.07,4.05,3.04,2.03,1.01,0.00
RESL,SF3,7.07,5.65,4.24,2.83,1.41,0.00
TEAM,SF4,5.48,4.38,3.29,2.19,1.10,0.00
PMAT,SF5,7.80,6.24,4.68,3.12,1.56,0.00
RELY,EM1,0.82,0.92,1.00,1.10,1.26,NaN
DATA,EM2,NaN,0.90,1.00,1.14,1.28,NaN
CPLX,EM3,0.73,0.87,1.00,1.17,1.34,1.74
RUSE,EM4,NaN,0.95,1.00,1.07,1.15,1.24


In [162]:
pd.read_csv('dados/params-cocomo2000-early.csv', index_col=0)

,Symbol,XL,VL,L,N,H,VH,XH
Driver,,,,,,,,
PERS,EM1,2.12,1.62,1.26,1.0,0.83,0.63,0.50
RCPX,EM2,0.49,0.60,0.83,1.0,1.33,1.91,2.72
PDIF,EM3,NaN,NaN,0.87,1.0,1.29,1.81,2.61
PREX,EM4,1.59,1.33,1.12,1.0,0.87,0.74,0.62
FCIL,EM5,1.43,1.30,1.10,1.0,0.87,0.73,0.62
RUSE,EM6,NaN,NaN,0.95,1.0,1.07,1.15,1.24
SCED,EM7,NaN,1.43,1.14,1.0,1.00,1.00,NaN


In [250]:
params_constantes = pd.read_csv('dados/params-cocomo2000-constants.csv', index_col=0)
params_constantes

,Baseline,Value
Constants,,
A,Effort,2.94
B,Effort,0.91
C,Schedule,3.67
D,Schedule,0.28


In [251]:
def matrix_interpolada(tabela_cocomo):
    valores_params = tabela_cocomo[tabela_cocomo.columns[1:]].values
    for i in range(valores_params.shape[0]):
        a = valores_params[i]
        nans = np.isnan(a)
        if nans.sum() > 0:
            s = np.linalg.lstsq(np.vstack([np.nonzero(~nans)[0], np.ones(len(a[~nans]))]).T, a[~nans], rcond=None)[0]
            valores_params[i, nans] = np.nonzero(nans)[0]*s[0] + s[1]
    return valores_params

valores_params = matrix_interpolada(params)
valores_params

array([[6.2  , 4.96 , 3.72 , 2.48 , 1.24 , 0.   ],
       [5.07 , 4.05 , 3.04 , 2.03 , 1.01 , 0.   ],
       [7.07 , 5.65 , 4.24 , 2.83 , 1.41 , 0.   ],
       [5.48 , 4.38 , 3.29 , 2.19 , 1.1  , 0.   ],
       [7.8  , 6.24 , 4.68 , 3.12 , 1.56 , 0.   ],
       [0.82 , 0.92 , 1.   , 1.1  , 1.26 , 1.338],
       [0.76 , 0.9  , 1.   , 1.14 , 1.28 , 1.4  ],
       [0.73 , 0.87 , 1.   , 1.17 , 1.34 , 1.74 ],
       [0.863, 0.95 , 1.   , 1.07 , 1.15 , 1.24 ],
       [0.81 , 0.91 , 1.   , 1.11 , 1.23 , 1.324],
       [0.533, 0.74 , 1.   , 1.11 , 1.29 , 1.63 ],
       [0.645, 0.795, 1.   , 1.05 , 1.17 , 1.46 ],
       [0.72 , 0.87 , 1.   , 1.15 , 1.3  , 1.44 ],
       [1.42 , 1.19 , 1.   , 0.85 , 0.71 , 0.506],
       [1.34 , 1.15 , 1.   , 0.88 , 0.76 , 0.597],
       [1.29 , 1.12 , 1.   , 0.9  , 0.81 , 0.67 ],
       [1.22 , 1.1  , 1.   , 0.88 , 0.81 , 0.69 ],
       [1.19 , 1.09 , 1.   , 0.91 , 0.85 , 0.75 ],
       [1.2  , 1.09 , 1.   , 0.91 , 0.84 , 0.738],
       [1.17 , 1.09 , 1.   , 0.

In [252]:
def gerar_pesos_rede(matriz_cocomo, quant_SF, dim_entrada, dim_inicial, dim_intermediaria, A, B):
    pesos_iniciais = np.zeros((dim_entrada, dim_inicial), dtype=np.float32)
    pesos_iniciais[0, 0] = 1
    pesos_iniciais[1:quant_SF*matriz_cocomo.shape[1]+1, 1] = np.ravel(matriz_cocomo[:quant_SF,:])/100
    pesos_iniciais[quant_SF*matriz_cocomo.shape[1]+1:, 2] = np.log(np.ravel(matriz_cocomo[quant_SF:,:]))
    polarizacao_inicial = np.array([0, B, np.log(A)], dtype=np.float32)
    pesos_log = np.zeros((dim_inicial, dim_intermediaria), dtype=np.float32)
    pesos_log[0, 0] = 1
    polarizacao_log = np.zeros(dim_intermediaria, dtype=np.float32)
    polarizacao_log[1] = 1
    pesos_recombina = np.zeros((dim_inicial, dim_intermediaria), dtype=np.float32)
    pesos_recombina[1:, :] = np.eye(dim_intermediaria)
    polarizacao_recombina = np.zeros(dim_intermediaria, dtype=np.float32)
    peso_exponencial = np.ones((1, 1))
    polarizacao_exponencial = np.zeros(1)
    pesos_rede = [pesos_iniciais, polarizacao_inicial, pesos_log, polarizacao_log, pesos_recombina, polarizacao_recombina, peso_exponencial, polarizacao_exponencial]
    return pesos_rede

quant_SF = params['Symbol'].str.contains('SF').sum()
dim_entrada = valores_params.size + 1
pesos_rede = gerar_pesos_rede(valores_params, quant_SF, dim_entrada, dim_inicial, dim_intermediaria, params_constantes.loc['A', 'Value'], params_constantes.loc['B', 'Value'])
pesos_rede

[array([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.062     ,  0.        ],
        [ 0.        ,  0.0496    ,  0.        ],
        [ 0.        ,  0.0372    ,  0.        ],
        [ 0.        ,  0.0248    ,  0.        ],
        [ 0.        ,  0.0124    ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0507    ,  0.        ],
        [ 0.        ,  0.0405    ,  0.        ],
        [ 0.        ,  0.0304    ,  0.        ],
        [ 0.        ,  0.0203    ,  0.        ],
        [ 0.        ,  0.0101    ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0707    ,  0.        ],
        [ 0.        ,  0.0565    ,  0.        ],
        [ 0.        ,  0.0424    ,  0.        ],
        [ 0.        ,  0.0283    ,  0.        ],
        [ 0.        ,  0.0141    ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0548    ,  0.        ],
        [ 0.        

### Instanciação e inicialização dos redes com os pesos

In [196]:
class AtivacaoLog(keras.layers.Layer):

    def __init__(self, dim_saida, inicializador='uniform', **kwargs):
        self.dim_saida = dim_saida
        self.inicializador = inicializador
        super(AtivacaoLog, self).__init__(**kwargs)

    def build(self, dim_entrada):
        self.pesos = self.add_weight(name='pesos',shape=(dim_entrada[1], self.dim_saida),initializer=self.inicializador,trainable=True)
        self.polarizacao = self.add_weight(name='polarizacao', shape=(self.dim_saida,), initializer='zeros')
        super(AtivacaoLog, self).build(dim_entrada)

    def call(self, dados):
        return keras.backend.log(keras.backend.bias_add(keras.backend.dot(dados, self.pesos), self.polarizacao, data_format='channels_last'))

    def compute_output_shape(self, dim_entrada):
        return (dim_entrada[0], self.dim_saida)

In [260]:
def constroi_modelo(dim_entrada, dim_inicial, dim_intermediaria):
    entrada = keras.layers.Input(shape=(dim_entrada,), dtype='float32', name='entrada')
    combina_linear = keras.layers.Dense(dim_inicial, activation='relu')(entrada)
    combina_log = AtivacaoLog(dim_intermediaria)(combina_linear)
    recombina_relu = keras.layers.Dense(dim_intermediaria, activation='relu')(combina_linear)
    produto_interno = keras.layers.dot([combina_log, recombina_relu], axes=1)
    saida = keras.layers.Dense(1, activation='relu')(produto_interno)
    return keras.models.Model(inputs=entrada, outputs=saida)

modelo_cocomo = constroi_modelo(dim_entrada, dim_inicial, dim_intermediaria)
modelo_cocomo.compile(loss='mse',optimizer='adam')
modelo_cocomo.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
entrada (InputLayer)            (None, 133)          0                                            
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 3)            402         entrada[0][0]                    
__________________________________________________________________________________________________
ativacao_log_9 (AtivacaoLog)    (None, 2)            8           dense_31[0][0]                   
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 2)            8           dense_31[0][0]                   
__________________________________________________________________________________________________
dot_8 (Dot

In [261]:
modelo_cocomo.set_weights(pesos_rede)
modelo_cocomo.get_weights()

[array([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.062     ,  0.        ],
        [ 0.        ,  0.0496    ,  0.        ],
        [ 0.        ,  0.0372    ,  0.        ],
        [ 0.        ,  0.0248    ,  0.        ],
        [ 0.        ,  0.0124    ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0507    ,  0.        ],
        [ 0.        ,  0.0405    ,  0.        ],
        [ 0.        ,  0.0304    ,  0.        ],
        [ 0.        ,  0.0203    ,  0.        ],
        [ 0.        ,  0.0101    ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0707    ,  0.        ],
        [ 0.        ,  0.0565    ,  0.        ],
        [ 0.        ,  0.0424    ,  0.        ],
        [ 0.        ,  0.0283    ,  0.        ],
        [ 0.        ,  0.0141    ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0548    ,  0.        ],
        [ 0.        

## Comparação entre os modelos

Exemplificação da equivalência matemática entre os modelos

### Formatação dos dados